# Использование фреймворка для решения задач RL

Рассмотрим работу одного из фреймворков для решения задач RL: `stable_baselines3` (далее SB3).
Для установки можно использовать команду в консоли

    pip install stable-baselines3[extra]

Документация по `stable_baselines3`:

    https://stable-baselines3.readthedocs.io/en/master/

Обучение ИНС в этом фреймворке происходит с помощью библиотеки `PyTorch`.

In [1]:
import stable_baselines3
stable_baselines3.__version__

'2.5.0'

In [2]:
import gymnasium as gym
gym.__version__

'1.0.0'

Ключевым объектом для работы с этой библиотекой является экземпляр класса `BaseAlgorithm`. Он содержит в себе стратегию агента в виде ИНС и целый ряд параметров. Конкретные алгоритмы являются наследниками этого класса. Список алгоритмов можно найти по ссылке:

https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

Рассмотрим пример на основе алгоритма PPO и среды `CartPole`.

In [9]:
# подгрузка алгоритма
from stable_baselines3 import PPO
# создание среды
env = gym.make("CartPole-v1")
# создание экземпляра алгоритма
model = PPO('MlpPolicy', env, verbose = 1) 

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


При создании экземпляра класса выбранного алгоритма требуется указать вид ИНС для стратегии. В данном случае это обычная полносвязная ИНС 'MlpPolicy' с параметрами по умолчанию. В дальнейших примерах будет показано, как можно задавать свою ИНС для алгоритма. 

Также требуется передать среду. Параметр `verbose = 1` означает, что при обучении на печать будет выводиться промежуточная информация.

При создании экземпляра алгоритма среда оборачивается в классы обёртки:
    `Monitor` - для слежения за выполнением,
    `DummyVecEnv` - обёртка с фейк-векторизацией.

Суть в том, что SB3 позволяет использовать агента одновременно в нескольких экземплярах среды. Это позволяет добавить скорости к обучению. При этом используется обёртка, которая векторизует эту среду. Далее, будет пример с векторизованной средой, однако, пока используем одну среду для обучения. И здесь используется обёртка по умолчанию `DummyVecEnv`.

Полный набор входных параметров для алгоритма можно вывести следующей командой.

In [9]:
PPO??

Init signature:
PPO(
    policy: Union[str, type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[dict[str, Any]]

Обучим модель методом `learn()`. Параметром можно указать число шагов в среде, использующихся для обучения.

In [9]:
model.learn(total_timesteps=25000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.2     |
|    ep_rew_mean     | 21.2     |
| time/              |          |
|    fps             | 1566     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 27.4        |
|    ep_rew_mean          | 27.4        |
| time/                   |             |
|    fps                  | 1104        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009800962 |
|    clip_fraction        | 0.111       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000796   |
|    learning_rate        | 0.

Можно не выводить промежуточную информацию при обучении, для этого можно указать `verbose = 0` (или ничего не указывать, параметр `verbose` равен 0 по умолчанию). Однако, для отслеживания прогресса можно в методе `learn()` указать параметр `progress_bar=True`.

In [11]:
model = PPO('MlpPolicy', env, verbose = 0) 
model.learn(total_timesteps=25000, progress_bar=True)

Output()

Для демонстрации работы агента можно запустить симуляцию. Для получения оптимального действия у обученной модели надо вызвать метод `predict()`. Для методов типа PPO, где стратегия стохастична по определению, при вызове метода `predict()` рекомендуется указывать параметр `deterministic=True`.

In [13]:
env_eval = gym.make("CartPole-v1", render_mode = "human")
env_eval = gym.wrappers.RecordEpisodeStatistics(env_eval)

state, _ = env_eval.reset()

while True:
    action, _ = model.predict(state, deterministic=True)
    state, reward, terminated, truncated, _ = env_eval.step(action)
    if terminated or truncated:
        break
    
print("Доход:", env_eval.return_queue[-1])

env.close()

Доход: 500.0


Сохранить или загрузить модель можно командами 

    model.save('filename')
    model = PPO.load('filename', env=env)

In [17]:
import os
save_path = os.path.join('savedModels', 'SB3_PPO_CartPole')
model.save(save_path)

Для оценки полученной стратегии на большом числе эпизодов можно использовать также встроенный метод `evaluate_policy()`, указав для него необходимые параметры. 

In [19]:
from stable_baselines3.common.evaluation import evaluate_policy

model = PPO.load(save_path, env=env)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)

print("Средний доход: ", mean_reward)
print("Стандартное отклонение: ", std_reward)
env.close()

D:\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Средний доход:  500.0
Стандартное отклонение:  0.0


Этот же метод можно использовать и для демонстрации работы агента.

In [13]:
env_eval = gym.make("CartPole-v1", render_mode = "human")
import os
from stable_baselines3.common.evaluation import evaluate_policy
save_path = os.path.join('savedModels', 'SB3_PPO_CartPole')
model = PPO.load(save_path, env=env)

evaluate_policy(model, env_eval, n_eval_episodes=1, render=True)
env.close()

D:\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


### Пример обучения с наблюдением процесса обучения через Tensorboard

Рассмотрим среду 'BipedalWalker-v3'. 

    https://gymnasium.farama.org/environments/box2d/bipedal_walker/

Это простая модель 4-суставного шагающего робота. Есть две версии:

     Обычная, со слегка неровной местностью,
     Сложная, с препятствиями.

Цель: набрать 300 очков за 1600 временных шагов.

In [3]:
env = gym.make('BipedalWalker-v3')
print(env.action_space)
print(env.observation_space)

Box(-1.0, 1.0, (4,), float32)
Box([-3.1415927 -5.        -5.        -5.        -3.1415927 -5.
 -3.1415927 -5.        -0.        -3.1415927 -5.        -3.1415927
 -5.        -0.        -1.        -1.        -1.        -1.
 -1.        -1.        -1.        -1.        -1.        -1.       ], [3.1415927 5.        5.        5.        3.1415927 5.        3.1415927
 5.        5.        3.1415927 5.        3.1415927 5.        5.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.       ], (24,), float32)


Запустим симуляцию со случайными действиями.

In [27]:
env = gym.make('BipedalWalker-v3', render_mode = 'human')
env = gym.wrappers.RecordEpisodeStatistics(env)

state, _ = env.reset()

while True:
    action = env.action_space.sample()
    state, reward, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        break
    
print("Доход:", env.return_queue[-1])
env.close()

Доход: -122.55451860004999


Это среда требует большего времени для обучения. За обучением удобно следить, например, с помощью методом библиотеки `Tensorboard`.

    pip install tensorboard

При создании модели в SB3 можно указать папку для создания tensorboard-логов.

In [11]:
env = gym.make('BipedalWalker-v3')

import os
logs_path = os.path.join('logs', 'ppo_bipedal_tensorboard')
model = PPO("MlpPolicy", env, tensorboard_log=logs_path)

Для запуска `tensorboard` надо в консоли в процессе обучения прописать специальную строчку, указав пусть к папке с логами.

Сначала построим путь к папке с логами.

In [19]:
os.getcwd() # получить путь к текущей папке, где лежит Python-ноутбук
full_logs_path = os.path.join(os.getcwd() ,'logs', 'ppo_bipedal_tensorboard')
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\ppo_bipedal_tensorboard"


Эту напечатанную строчку надо запустить в консоли после запуска обучения и перейти по ссылке которая там возникнет ( http://localhost:6006/)

Запустим обучение.

In [21]:
model.learn(total_timesteps=2_000_000, progress_bar=True)

Output()

KeyboardInterrupt: 

In [81]:
save_path =  os.path.join('savedModels', 'SB3_PPO_BiPedal')
model.save(save_path)
print(save_path)

savedModels\SB3_PPO_BiPedal


In [13]:
from stable_baselines3.common.evaluation import evaluate_policy

save_path='savedModels/SB3_PPO_BiPedal'

env = gym.make('BipedalWalker-v3')
model = PPO.load(save_path, env=env)

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=False)
print("Средний доход: ", mean_reward)
print("Стандартное отклонение: ", std_reward)
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


D:\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Средний доход:  307.90262765631195
Стандартное отклонение:  0.5292323403922334


In [15]:
env = gym.make('BipedalWalker-v3', render_mode = 'human')
evaluate_policy(model, env, n_eval_episodes=1, render=True)

(307.1989354356192, 0.0)

Добраться до используемой ИНС внутри модели можно, обратившись к полю `model.policy`. Архитектуру ИНС можно отобразить с помощью библиотеки `torchsummary`.

In [17]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=24, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=24, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=4, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [39]:
from torchsummary import summary

summary(model.policy, input_size=(1,24,))

----------------------------------------------------------------

Layer (type)               Output Shape         Param #

================================================================

Flatten-1                   [-1, 24]               0

FlattenExtractor-2                   [-1, 24]               0

...

Input size (MB): 0.00

Forward/backward pass size (MB): 0.03

Params size (MB): 0.05

Estimated Total Size (MB): 0.07

----------------------------------------------------------------

### Обратная связь при обучении (Callbacks)

В ходе обучения наибоее важный параметр, который надо отслеживать - 
это средний доход за эпизод. При достижении определённой величины дохода можно остановить обучение. 
Для того, чтобы настроить отслеживание среднего дохода за эпизод и прекращение обучения в библиотеке `SB3` есть специальные методы: `callback functions`.

С их помощью можно также можно сохранять модель по ходу обучения при выполнении некоторых условий и тд.

Создадим два callback-метода:

    1. Остановить обучение по достижении цели
    2. Сохранить модель по достижении нового лучшего результата

Для примера рассмотрим среду `CartPole-v1`.

In [19]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_PPO_CartPole')
save_path_best = os.path.join('savedModels', 'SB3_PPO_CartPole_best')
log_path = os.path.join('logs', 'SB3_PPO_CartPole')

env = gym.make("CartPole-v1")

In [21]:
# создание callback-методов
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=400, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=5000, 
                             best_model_save_path=save_path_best, 
                             verbose=1)

То есть каждые 5000 временных шагов будет вызываться EvalCallback() для проверки качества текущей модели. Если модель стала лучше, то вызывается `StopTrainingOnRewardThreshold()` для проверки того, стало ли качество модели выше, чем целевое значение. 

In [23]:
model = PPO('MlpPolicy', env, tensorboard_log=log_path)
model.learn(total_timesteps=20000, callback=eval_callback, progress_bar=True)

Output()

Eval num_timesteps=5000, episode_reward=332.20 +/- 119.24

Episode length: 332.20 +/- 119.24

New best mean reward!

Eval num_timesteps=10000, episode_reward=267.00 +/- 122.31

Episode length: 267.00 +/- 122.31

Eval num_timesteps=15000, episode_reward=464.00 +/- 72.00

Episode length: 464.00 +/- 72.00

New best mean reward!

Stopping training because the mean reward 464.00  is above the threshold 400

In [18]:
model.save(save_path)

In [22]:
load_path_best = os.path.join(save_path_best,"best_model")
load_path_best

'savedModels\\SB3_PPO_CartPole_best\\best_model'

In [26]:
model = PPO.load(load_path_best, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [25]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)

print("Средний доход: ", mean_reward)
print("Стандартное отклонение: ", std_reward)
env.close()

Средний доход:  455.46
Стандартное отклонение:  68.86950268442483


### Пример

Рассмотрим среду "Pendulum-v1" и применим для обучения агента метод PPO.

https://www.gymlibrary.dev/environments/classic_control/pendulum/

In [27]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [63]:
env = gym.make("Pendulum-v1", render_mode = 'human')
env = gym.wrappers.RecordEpisodeStatistics(env)

state, _ = env.reset()

while True:
    action = env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break
    
print("Доход:", env.return_queue[-1])

env.close()

Доход: -1029.790284745965


In [29]:
import os
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_PPO_Pendulum')
save_path_best = os.path.join('savedModels', 'SB3_PPO_Pendulum_best')
log_path = os.path.join('logs', 'SB3_PPO_Pendulum')

env = gym.make("Pendulum-v1")

# создание callback-методов
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=-170, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path_best, 
                             verbose=1)

model = PPO('MlpPolicy', env, tensorboard_log=log_path)

In [31]:
from torchsummary import summary

summary(model.policy, input_size=(1,3,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                    [-1, 3]               0
  FlattenExtractor-2                    [-1, 3]               0
            Linear-3                   [-1, 64]             256
              Tanh-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              Tanh-6                   [-1, 64]               0
            Linear-7                   [-1, 64]             256
              Tanh-8                   [-1, 64]               0
            Linear-9                   [-1, 64]           4,160
             Tanh-10                   [-1, 64]               0
     MlpExtractor-11       [[-1, 64], [-1, 64]]               0
           Linear-12                    [-1, 1]              65
           Linear-13                    [-1, 1]              65
Total params: 8,962
Trainable params: 8

Путь к логам Tensorboard.

In [37]:
full_logs_path = os.path.join(os.getcwd() ,log_path)
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\SB3_PPO_Pendulum"


In [61]:
model.learn(total_timesteps=200000, callback=eval_callback, progress_bar=True)

Output()

Eval num_timesteps=10000, episode_reward=-1155.75 +/- 150.93

Episode length: 200.00 +/- 0.00

New best mean reward!

Eval num_timesteps=20000, episode_reward=-1407.20 +/- 224.11

...

New best mean reward!

Eval num_timesteps=70000, episode_reward=-1091.30 +/- 332.43

Episode length: 200.00 +/- 0.00

New best mean reward!

KeyboardInterrupt: 

In [75]:
model.save(save_path)

In [33]:
load_path_best = os.path.join(save_path_best,"best_model")
env = gym.make("Pendulum-v1")
model = PPO.load(load_path_best, env=env)

In [35]:
env = gym.make("Pendulum-v1", render_mode = 'human')
gain, _ = evaluate_policy(model, env, n_eval_episodes=1, render=True)
print(gain)
env.close()

-721.793465230614


### Смена дефолтной архитектуры сети

Архитертура ИНС, которая по умолчанию задаётся при вызове "MlpPolicy" у алгоритма обучения, может быть изменена.

Для метода PPO начальные слои ИНС отвечают за извлечение признаков из состояний на входе, эти слои общие для актора и критика. После этих слоев сеть делится на две части: одна для актора, другая для критика. 

Для простых по структуре состояний (например, в виде векторов) слои с извлечением признаков пусты. Для смены ахритектуры дальнейших слоёв можно использовать синтаксис

    policy_kwargs = dict(net_arch=dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128]))
    model = PPO('MlpPolicy', env, policy_kwargs=policy_kwargs)

То есть для каждой ветки ИНС будет создано по 4 слоя по 128 нейронов. См. подробности

https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html

Применим рассмотренные методы для более сложной задачи. 

### Пример со сменой архитектуры ИНС

Рассмотрим среду "InvertedDoublePendulum-v5".

In [41]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [78]:
env = gym.make("InvertedDoublePendulum-v5", render_mode = 'human')
env = gym.wrappers.RecordEpisodeStatistics(env)

state, _ = env.reset()

while True:
    action = env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break
    
print("Доход:", env.return_queue[-1])

env.close()

Доход: 44.589997605297434

In [43]:
import os
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_PPO_InvPendulum')
save_path_best = os.path.join('savedModels', 'SB3_PPO_InvPendulum_best')
log_path = os.path.join('logs', 'SB3_PPO_InvPendulum')

env = gym.make("InvertedDoublePendulum-v5")

# создание callback
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=3000, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path_best, 
                             verbose=1)

#model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

policy_kwargs = dict(net_arch = dict(pi=[96, 96, 96], vf=[96, 96, 96]))
model = PPO('MlpPolicy', env, policy_kwargs=policy_kwargs, tensorboard_log=log_path)

In [47]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=9, out_features=96, bias=True)
      (1): Tanh()
      (2): Linear(in_features=96, out_features=96, bias=True)
      (3): Tanh()
      (4): Linear(in_features=96, out_features=96, bias=True)
      (5): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=9, out_features=96, bias=True)
      (1): Tanh()
      (2): Linear(in_features=96, out_features=96, bias=True)
      (3): Tanh()
      (4): Linear(in_features=96, out_features=96, bias=True)
      (5): Tanh()
    )
  )
  (action_net): Linear(in_features=96, out_features=1, bias=True)
  (value_net): Linear(in_features=96, out

In [45]:
summary(model.policy, input_size=(1,9,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                    [-1, 9]               0
  FlattenExtractor-2                    [-1, 9]               0
            Linear-3                   [-1, 96]             960
              Tanh-4                   [-1, 96]               0
            Linear-5                   [-1, 96]           9,312
              Tanh-6                   [-1, 96]               0
            Linear-7                   [-1, 96]           9,312
              Tanh-8                   [-1, 96]               0
            Linear-9                   [-1, 96]             960
             Tanh-10                   [-1, 96]               0
           Linear-11                   [-1, 96]           9,312
             Tanh-12                   [-1, 96]               0
           Linear-13                   [-1, 96]           9,312
             Tanh-14                   

In [49]:
full_logs_path = os.path.join(os.getcwd() ,log_path)
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\SB3_PPO_InvPendulum"


In [51]:
model.learn(total_timesteps=200000, callback=eval_callback, progress_bar=True)

Output()

Eval num_timesteps=10000, episode_reward=221.98 +/- 114.47

Episode length: 25.00 +/- 12.21

New best mean reward!

Eval num_timesteps=20000, episode_reward=329.19 +/- 27.70

...

Episode length: 175.40 +/- 47.66

Eval num_timesteps=120000, episode_reward=4541.09 +/- 810.03

Episode length: 488.40 +/- 86.85

New best mean reward!

Stopping training because the mean reward 4541.09  is above the threshold 3000

In [100]:
model.save(save_path)

In [86]:
load_path = os.path.join(save_path_best, 'best_model')

model = PPO.load(load_path, env=env)

In [53]:
env = gym.make("InvertedDoublePendulum-v5", render_mode = 'human')
gain, _ = evaluate_policy(model, env, n_eval_episodes=1, render=True)
print(gain)
env.close()

5656.374836444855


In [108]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)

print("Средний доход: ", mean_reward)
print("Стандартное отклонение: ", std_reward)
env.close()

Средний доход:  5789.755002372265
Стандартное отклонение:  2903.251204989814


### Векторизованные среды

Векторизация сред позволяет в фреймворке `SB3` запустить A2C метод, где несколько агентов (Workers) собирают данные. Для этого надо создать векторизованную среду методом `make_vec_env()`.

Сравним работу векторизованной среды и обычной.

In [55]:
import os
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_A2C_CartPole')
log_path = os.path.join('logs', 'SB3_A2C_CartPole')

In [57]:
env = gym.make("CartPole-v1")
model = A2C('MlpPolicy', env, tensorboard_log=log_path)

In [59]:
# векторизованная среда
vec_env = make_vec_env("CartPole-v1", n_envs=4)
model_vec = A2C("MlpPolicy", vec_env, tensorboard_log=log_path)

In [61]:
full_logs_path = os.path.join(os.getcwd() ,log_path)
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\SB3_A2C_CartPole"


In [63]:
model.learn(total_timesteps=25000, progress_bar=True)

Output()

In [65]:
model_vec.learn(total_timesteps=25000, progress_bar=True)

Output()

In [13]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)
print(mean_reward)

D:\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


34.84


In [15]:
mean_reward, std_reward = evaluate_policy(model_vec, env, n_eval_episodes=100, render=False)
print(mean_reward)

468.56


### Ещё несколько примеров сред

In [115]:
import os
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_A2C_InvertedDoublePendulum')
log_path = os.path.join('logs', 'SB3_A2C_InvertedDoublePendulum')

env = gym.make("InvertedDoublePendulum-v5")
model = A2C('MlpPolicy', env, tensorboard_log=log_path)

# векторизованная среда
vec_env = make_vec_env("InvertedDoublePendulum-v5", n_envs=4)
model_vec = A2C("MlpPolicy", vec_env, tensorboard_log=log_path)

In [117]:
full_logs_path = os.path.join(os.getcwd() ,log_path)
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\SB3_A2C_InvertedDoublePendulum"


In [119]:
model.learn(total_timesteps=100000, progress_bar=True)

Output()

In [120]:
model_vec.learn(total_timesteps=100000, progress_bar=True)

Output()

In [124]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)
print(mean_reward)

443.04261385083197


In [125]:
mean_reward, std_reward = evaluate_policy(model_vec, env, n_eval_episodes=100, render=False)
print(mean_reward)

481.7737252724171


Пример со средой `MountainCar`, где действия не дискретны, а непрерывны.

In [143]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_PPO_MountainCarContinuous')
log_path = os.path.join('logs', 'SB3_PPO_MountainCarContinuous')

env = gym.make("MountainCarContinuous-v0")
model = PPO('MlpPolicy', env, tensorboard_log=log_path)

# векторизованная среда
vec_env = make_vec_env("MountainCarContinuous-v0", n_envs=4)
model_vec = PPO("MlpPolicy", vec_env, tensorboard_log=log_path)

In [145]:
model.learn(total_timesteps=100000, progress_bar=True)

Output()

In [146]:
model_vec.learn(total_timesteps=100000, progress_bar=True)

Output()

In [147]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)
print(mean_reward)

-9.670974249588583e-06


In [148]:
mean_reward, std_reward = evaluate_policy(model_vec, env, n_eval_episodes=100, render=False)
print(mean_reward)

-0.001369864273592881


In [153]:
env = gym.make("MountainCarContinuous-v0", render_mode = 'human')
gain, _ = evaluate_policy(model_vec, env, n_eval_episodes=1, render=True)
print(gain)

KeyboardInterrupt: 